<a href="https://colab.research.google.com/github/MinZhenTay/Recipe-Recommendation/blob/main/Recipe_Recommendation_using_LDA_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Recipe Recommendation System using LDA Algorithm**

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
# Upload the Dataset
recipes = pd.read_csv('/Users/60197/Desktop/Research Project/FinalRecipeData.csv')

In [3]:
recipes.head()

,Unnamed: 0,recipe_name,ingredients,cooking_directions,nutritions
0,0,air fried raspberry brie bites,"['1 (8 ounce) round brie cheese', '1 (8 ounce)...",['\nslice the brie into squares about ½ inch t...,"[{'calories': '252'}, {'fat': '16g '}, {'carbs..."
1,1,spicy chicken jerky in the air fryer,"['2 (5 ounce) boneless chicken breasts, cut in...","['\ncombine chicken strips, marinade, and caju...","[{'calories': '54'}, {'fat': '1g '}, {'carbs':..."
2,2,air fryer scrambled eggs on the go,"[' cooking spray', '2 large eggs, beaten', ' ...",['\nlightly spray a ramekin dish with non-stic...,"[{'calories': '143'}, {'fat': '10g '}, {'carbs..."
3,3,basic air fryer hot dogs,"['4 hot dog buns', '4 hot dogs']",['\npreheat an air fryer to 400 degrees f (200...,"[{'calories': '269'}, {'fat': '15g '}, {'carbs..."
4,4,air fryer potato wedges,"['2 medium russet potatoes', '1 ½ tablespoons ...",['\npreheat an air fryer to 400 degrees f (200...,"[{'calories': '129'}, {'fat': '5g '}, {'carbs'..."


In [4]:
# Check if there is any missing value
missing_values = recipes.isnull().sum()
print(missing_values)

Unnamed: 0            0
recipe_name           0
ingredients           0
cooking_directions    0
nutritions            0
dtype: int64


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\60197\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# Remove stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [7]:
# Preprocess data in the 'ingredients' column
recipes['ingredients'] = recipes['ingredients'].str.lower()
recipes['ingredients'] = recipes['ingredients'].str.replace('cup','')
recipes['ingredients'] = recipes['ingredients'].str.replace(r'[¼½⅓]','')
recipes['ingredients'] = recipes['ingredients'].str.replace(r'^','')

C:\Users\60197\AppData\Local\Temp/ipykernel_14796/544913623.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['ingredients'] = recipes['ingredients'].str.replace(r'[¼½⅓]','')
C:\Users\60197\AppData\Local\Temp/ipykernel_14796/544913623.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  recipes['ingredients'] = recipes['ingredients'].str.replace(r'^','')


In [8]:
recipes['ingredients']

0        ['1 (8 ounce) round brie cheese', '1 (8 ounce)...
1        ['2 (5 ounce) boneless chicken breasts, cut in...
2        ['  cooking spray', '2 large eggs, beaten', ' ...
3                       ['4  hot dog buns', '4  hot dogs']
4        ['2 medium russet potatoes', '1  tablespoons o...
                               ...                        
49538    green bell pepperred bell pepperonioncelerylar...
49539    olive oilgarliclarge onionlarge green bell pep...
49540    sliced peaches drainedscoops vanilla ice cream...
49541    butterhabanero peppersfresh peachesbrown sugar...
49542    olive oilbulb garlictomatoes choppeddiced toma...
Name: ingredients, Length: 49543, dtype: object

In [9]:
# Join text for 'recipe_name', 'cooking_directions' & 'ingredients'
all_text = recipes['recipe_name'] + ' ' + recipes['ingredients'] + ' ' + recipes['cooking_directions']
all_text[0]

"air fried raspberry brie bites ['1 (8 ounce) round brie cheese', '1 (8 ounce) package phyllo dough, thawed', '1  raspberry jam', '1  butter, melted', '2 tablespoons honey', '  flakey salt to taste'] ['\\nslice the brie into squares about ½ inch thick and 1 to 1 ½ inches wide.\\n', '\\nroll out 1 sheet of phyllo dough vertically on a large work surface. brush the bottom half of the dough with melted butter, then fold the top half over onto itself.\\n', '\\nplace 1 piece of cheese in the bottom center of the dough, about 2 inches from the edges. spoon about 2 teaspoons raspberry jam onto the cheese. brush melted butter around the rest of the dough.\\n', '\\nfold in the bottom 2 inches of dough over the cheese, then fold in both sides. brush more butter onto the remaining dough, then carefully roll the filling upwards until it is wrapped fully. brush more butter on top and set into the air fryer try.\\n', '\\nset air fryer to 375 degrees f (190 degrees c).\\n', '\\nair fry in batches to 

In [10]:
# Apply clean_text function
import string
import re

def clean_text(documents):
    cleaned_text = []
    for doc in documents:
        doc = doc.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
        doc = re.sub(r'\d+', '', doc) # Remove digits
        doc = doc.replace('\n',' ') # Remove new lines
        doc = doc.strip() # Remove leading white space
        doc = re.sub(' +', ' ', doc) # Remove multiple white spaces
        cleaned_text.append(doc)
    return cleaned_text

# Clean the text
cleaned_text = clean_text(all_text)

In [11]:
cleaned_text[2]

'air fryer scrambled eggs on the go cooking spray large eggs beaten salt and freshly ground black pepper to taste nlightly spray a ramekin dish with nonstick cooking spray pour in beaten eggsn nset air fryer to degrees f degrees c and place the ramekin into the air fryer basketn nair fry for minutes stir egg mixture and cook for minutes more fluff with a fork and season to taste with salt and pepper grab and gon'

In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

In [13]:
# Apply stopwords removal & word lemmatization
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [14]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\60197\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\60197\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [15]:
processed_text = [preprocess(text) for text in cleaned_text]

In [16]:
processed_text[2 ]

['fryer',
 'scrambl',
 'egg',
 'cook',
 'spray',
 'larg',
 'egg',
 'beat',
 'salt',
 'fresh',
 'grind',
 'black',
 'pepper',
 'tast',
 'nlight',
 'spray',
 'ramekin',
 'dish',
 'nonstick',
 'cook',
 'spray',
 'pour',
 'beat',
 'eggsn',
 'nset',
 'fryer',
 'degre',
 'degre',
 'place',
 'ramekin',
 'fryer',
 'basketn',
 'nair',
 'minut',
 'stir',
 'mixtur',
 'cook',
 'minut',
 'fluff',
 'fork',
 'season',
 'tast',
 'salt',
 'pepper',
 'grab']

In [17]:
dictionary = gensim.corpora.Dictionary(processed_text)
corpus = [dictionary.doc2bow(text) for text in processed_text]

In [18]:
# Apply LDA algorithm
lda_model = gensim.models.LdaModel(corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 10)
term_topic_weights = lda_model.get_topics()

In [19]:
# Create function for recommendation
def get_recommendations(ingredients, processed_text, dictionary, lda_model):
    ingredients_bow = dictionary.doc2bow(preprocess(ingredients))
    lda_topics = lda_model.get_document_topics(ingredients_bow)
    lda_topics = sorted(lda_topics, key=lambda x: x[1], reverse=True)
    top_topic = lda_topics[0][0]
    
    recommendations = []
    for index, row in enumerate(processed_text):
        topic_probs = lda_model.get_document_topics(dictionary.doc2bow(row))
        for topic in topic_probs:
            if topic[0] == top_topic and topic[1] > 0.5:
                recommendations.append(index)
    
    return recommendations

In [20]:
# Include the ingredient(s) input
ingredients = "pasta, potato"
recommendations = get_recommendations(ingredients, processed_text, dictionary, lda_model)

In [21]:
recipe_names = [recipes['recipe_name'][index] for index in recommendations]

In [22]:
# List of the titles for recommended recipes
recipe_names[:5]

['individual green bean casseroles',
 'mushroom stroganoff with creamy miso pasta',
 'curry mashed potatoes',
 'ham potato and cabbage stew',
 'easy caponata']

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\60197\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
# Tokenize the input ingredients text
input_ingredients_tokens = word_tokenize(ingredients)

# Clean the input ingredients text
cleaned_input_ingredients = [word for word in input_ingredients_tokens if word.isalnum()]

In [25]:
# Create the input_bow variable
input_bow = dictionary.doc2bow(cleaned_input_ingredients)

In [26]:
from gensim import matutils

# Get the topic distribution of the input ingredients
input_topics = lda_model.get_document_topics(input_bow)

# Create empty list to store the similarity scores
similarity_scores = []

# For each document in the corpus
for doc in corpus:
    # Get the topic distribution of the document
    doc_topics = lda_model.get_document_topics(doc)
    # Calculate the similarity score between the ingredients input and the document topics
    similarity_scores.append(matutils.cossim(input_topics, doc_topics))

In [27]:
num_recommendations = 5

In [28]:
# Get the indexes of the most similar recipes
most_similar_indexes = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i])[-num_recommendations:]

# Create empty list to store the recommended recipe names
recommended_recipe_names = []

# For each index in the most similar indexes
for index in most_similar_indexes:
# Append the recipe name to the list of recommended recipe names
    recommended_recipe_names.append(recipes.iloc[index]["recipe_name"])

# Create empty list to store the recommended recipes
recommended_recipes = []

# For each recommended recipe name
for recipe_name in recommended_recipe_names:
# Find the recipe in the original dataframe
    recipe = recipes[recipes["recipe_name"] == recipe_name]
# Append the recipe to the list of recommended recipes
    recommended_recipes.append(recipe)

# Print the details of recommended recipes
for recipe in recommended_recipes:
    print(recipe)

       Unnamed: 0                   recipe_name  \
38553       32449  double butternut squash soup   

                                             ingredients  \
38553  unsalted butteralmondsvegetable broth dividedb...   

                                      cooking_directions  \
38553  {'directions': u'prep\n20 m\ncook\n25 m\nready...   

                                              nutritions  
38553  {u'niacin': {u'hascompletedata': false, u'name...  
       Unnamed: 0                 recipe_name  \
11356       14443  sour cream mashed potatoes   

                                             ingredients  \
11356  ['3 pounds yukon gold potatoes, peeled and cut...   

                                      cooking_directions  \
11356  ['\nplace potatoes and 1 1/2 tablespoons of th...   

                                              nutritions  
11356  [{'calories': '529'}, {'fat': '37g '}, {'carbs...  
       Unnamed: 0                recipe_name  \
38681       32605  zucchini so

In [29]:
recommended_recipes_data = recipes[recipes.index.isin(recommended_recipes)]

In [30]:
# Print the details of recommended recipes
for recipe in recommended_recipes:
    print(recipe.to_string())
for index, recipe in recommended_recipes_data.iterrows():
    print(recipe['recipe_name'])
    print(recipe['ingredients'])
    print(recipe['cooking_directions'])
    print(recipe['nutritions'])

       Unnamed: 0                   recipe_name                                                                                                                                                   ingredients                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           